In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'


In [2]:
years = range(2013, 2015)
campeonatos = ['Serie_A']

for year in years:
    for campeonato in campeonatos:
        print(year, campeonato)
        df_jogos_2014 = pd.read_json(f'../Data/{campeonato}_{year}_games.json')
        df_jogos_2014_ = df_jogos_2014.T


        for index, row in df_jogos_2014_.iterrows():
            if index-1 < len(df_jogos_2014_):
                processor = FootballDataProcessor(df_jogos_2014_, index-1)
                final_df = processor.process(data_folder, campeonato, year)
                final_df.rename(columns={'player_id': 'player'}, inplace=True)

                final_df['player'] = final_df['player'].astype('string')
                id_players['player'] = id_players['player'].astype('string')

                final_df_ = pd.merge(final_df, id_players, on='player', how='left')
                


                team_1_name = df_jogos_2014_.iloc[index-1, 0]
                team_2_name = df_jogos_2014_.iloc[index-1, 1]

                # Criar uma instância de FootballMatchRating
                rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

                updated_match_data = rating_calculator.update_ratings()

                teams_over_9_players = updated_match_data[updated_match_data['Minutes Played'] == 90] \
                                        .groupby('team').filter(lambda x: len(x) > 10)
                if len(teams_over_9_players) != 0:
                    print(team_1_name, team_2_name, index-1)
                    continue

                updated_match_data['player'] = updated_match_data['player'].astype('string')

                updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
                updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

                updated_values = id_players[['player']].merge(
                    updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
                    on='player',
                    how='left'
                )

                # Atualizar apenas as linhas corespondentes em id_players

                id_players.update(updated_values)
            else:
                print(f"Índice {index-1} fora dos limites para o ano {year}")
                continue


id_players


2013 Serie_A
Atlético Mineiro / MG Grêmio / RS 17
Corinthians / SP Atlético Mineiro / MG 62
Atlético Mineiro / MG Athletico Paranaense / PR 97
Fluminense / RJ Flamengo / RJ 121
São Paulo / SP Flamengo / RJ 332
2014 Serie_A
Sport / PE Botafogo / RJ 91
Botafogo / RJ Figueirense / SC 340


,player,rating,age,games_played,player_name,time_jogador
0,430883,1500.000000,25,50,NaN,NaN
1,425934,1500.000000,25,50,NaN,NaN
2,156447,1500.000000,25,50,NaN,NaN
3,321556,1477.616578,25,53,11Rafael Rafael Diniz Alves e...,Ponte Preta / SP
4,395895,1500.000000,25,50,NaN,NaN
...,...,...,...,...,...,...
14892,438589,1500.000000,25,50,NaN,NaN
14893,375543,1500.000000,25,50,NaN,NaN
14894,311421,1500.000000,25,50,NaN,NaN
14895,379789,1500.000000,25,50,NaN,NaN


In [3]:
x = id_players.sort_values(by='rating', ascending=False)
x.to_excel('teste.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
9324,129292,1666.387044,25,122,1Fabio Fabio Deivson Lopes ...,Cruzeiro / MG
10896,163518,1666.085956,25,106,19Nilton Nilton Ferreira Junior,Cruzeiro / MG
1318,191616,1649.784127,25,109,28Ricardo Go ... Ricardo Goulart Pereira,Cruzeiro / MG
8012,166647,1649.428589,25,92,4Bruno Rodr ... Bruno Rodrigo Fenelo ...,Cruzeiro / MG
9266,177749,1646.492595,25,102,26Dede Anderson Vital da Silva,Cruzeiro / MG
...,...,...,...,...,...,...
646,166491,1365.447827,25,80,88Derley Wanderley de Jesus Sousa,Athletico Paranaense / PR
11603,173742,1365.355759,25,73,6Willian William Augusto Alve...,Náutico / PE
1719,135462,1342.602490,25,80,1Ricardo Ricardo Ferreira Berna,Náutico / PE
12169,177998,1336.815655,25,75,15Bruno Bruno Bairros Collaco,Chapecoense / SC
